In [ ]:
# Install required Google Cloud packages (commented out as these are typically one-time setup commands)
!pip install gcloud
!gcloud auth application-default login

# Import necessary Python libraries
import pandas as pd                # Data manipulation and analysis
import numpy as np                 # Numerical computing
import time                        # Time-related functions
import os                          # Operating system interfaces
import pandas_gbq                  # Pandas integration with BigQuery
from google.cloud import bigquery  # BigQuery client library
import glob                        # File path pattern matching
import openpyxl                    # Excel file handling
import csv                         # CSV file handling
import re                          # Regular expressions

# Note: The actual imports remain exactly as in the original code

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.4/454.4 kB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gcloud: filename=gcloud-0.18.3-py3-none-any.whl size=602927 sha256=19e4cf4d86d02affde2fa437bf961bd30ba3582128eac747b39fc3e3b1f6dc69
  Stored in directory: /root/.cache/pip/wheels/2a/62/75/3d74209bfebb8805823ae74afa28653aa1ea76d8b5a9d741ff
Successfully built gcloud
Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=fyYVJd1wRZAi7nMJS4pRVTVYNq1Qaf&prompt=consent&token_usage=remote&access_type=offline&code_chal

## Aposentadoria gráfico 2

In [ ]:
# Import necessary libraries
import pandas as pd
import pandas_gbq
import re

# Define the SQL query to select civil retiree data from a BigQuery table.
query = """
        SELECT ANO as ano, MES as mes,  Id_SERVIDOR_PORTAL as id_servidor, TIPO_APOSENTADORIA as tipo_aposentadoria, DATA_APOSENTADORIA as data_aposentadoria, DATA_DIPLOMA_INGRESSO_SERVICOPUBLICO as data_diploma_ingresso_servicopublico,
            CASE WHEN ORGSUP_LOTACAO ="Sem informação" THEN ORG_LOTACAO ELSE ORGSUP_LOTACAO
            END as org_lotacao, 'REGIME JURIDICO UNICO' as regime_juridico,
            1 as total FROM `repositoriodedadosgpsp.portal_transparencia_cgu.2024_11_siape_aposentados_cadastro`
        """
# Execute the query and load the result into a pandas DataFrame.
df = pandas_gbq.read_gbq(query, project_id='repositoriodedadosgpsp')
# Display the initial DataFrame.
df

# Check for null values in the 'data_aposentadoria' column.
aposentadoria = df['data_aposentadoria'].isnull()
# Count the number of null and non-null values for retirement date.
aposentadoria.value_counts()

# Check for null values in the 'data_diploma_ingresso_servicopublico' column.
ingresso = df['data_diploma_ingresso_servicopublico'].isnull()
# Count the number of null and non-null values for public service entry date.
ingresso.value_counts()

# Drop rows with any missing values in any column.
df = df.dropna()
# Display the DataFrame after dropping nulls.
df # Nulls in retirement date = 803; Nulls in entry date = 5989. The sum is the number of dropped rows (6792).

# Convert the 'data_aposentadoria' column to datetime objects.
df['data_aposentadoria'] = pd.to_datetime(df['data_aposentadoria'], dayfirst=True, format= "%d/%m/%Y") #transforming into date

# Convert the 'data_diploma_ingresso_servicopublico' column to datetime objects.
df['data_diploma_ingresso_servicopublico'] = pd.to_datetime(df['data_diploma_ingresso_servicopublico'], dayfirst=True, format= "%d/%m/%Y") #transforming into date

# Extract the year from the 'data_aposentadoria' column.
df['ano_aposentadoria'] = df['data_aposentadoria'].dt.year #getting only the year

# Extract the year from the 'data_diploma_ingresso_servicopublico' column.
df['ano_ingresso'] = df['data_diploma_ingresso_servicopublico'].dt.year #getting only the year

# Convert the values in the 'ano_ingresso' column to integers.
df['ano_ingresso']= df['ano_ingresso'].astype(int) #transforming the column values to integer

# Convert the values in the 'ano_aposentadoria' column to integers.
df['ano_aposentadoria'] = df['ano_aposentadoria'].astype(int) #transforming the column values to integer

# Define bin edges for creating decades.
limites = [0, 1959, 1969, 1979, 1989, 1999, 2009, 2019, 2024]
# Define labels for the decade bins.
categorias = [1950, 1960, 1970, 1980, 1990, 2000, 2010, 2020]

# Create a new column by categorizing the 'ano_aposentadoria' into decades.
df['decada_aposentadoria'] = pd.cut(df['ano_aposentadoria'], bins=limites, labels=categorias)

# Select and reorder columns for the DataFrame.
df = df[['ano', 'mes', 'tipo_aposentadoria', 'data_aposentadoria', 'ano_aposentadoria', 'decada_aposentadoria', 'data_diploma_ingresso_servicopublico', 'ano_ingresso', 'org_lotacao', 'regime_juridico', 'total']]
# Display the DataFrame.
df #411709 rows

# Calculate the contribution time by subtracting the entry year from the retirement year.
df['tempo_contribuicao_servico_publico'] = df['ano_aposentadoria'] - df['ano_ingresso']
# Display the new 'tempo_contribuicao_servico_publico' column.
df['tempo_contribuicao_servico_publico']

# Create a DataFrame with rows where contribution time is zero.
df_zeros = df[df['tempo_contribuicao_servico_publico']==0] #1338 rows
# Display the zero-contribution DataFrame.
df_zeros

# Create a DataFrame with rows where contribution time is negative.
menor_zero = df[df['tempo_contribuicao_servico_publico']< 0] #242 rows
# Display the negative-contribution DataFrame.
menor_zero

# Filter the main DataFrame to keep only rows with a positive contribution time.
df = df[df['tempo_contribuicao_servico_publico'] > 0]
# Display the unique values of contribution time to verify the filtering.
df['tempo_contribuicao_servico_publico'].unique()

# Convert the 'decada_aposentadoria' column to an integer type.
df['decada_aposentadoria'] = df['decada_aposentadoria'].astype(int)

# Define a function to rename and format retirement types using regular expressions.
def renomear(var_aposentadoria):
    var_aposentadoria = re.sub(r"\bAPOSENTADORIA POR INVALIDEZ\b", "Aposentadoria por invalidez", var_aposentadoria)
    var_aposentadoria = re.sub(r"\bAPOSENTADORIA VOLUNTARIA\b", "Aposentadoria voluntária", var_aposentadoria)
    var_aposentadoria = re.sub(r"\bAPOSENTADORIA COMPULSORIA\b", "Aposentadoria compulsória", var_aposentadoria)
    var_aposentadoria = re.sub(r"\bAPOSENTADORIA POR INCAPACIDADE\b", "Aposentadoria por incapacidade", var_aposentadoria)
    var_aposentadoria = re.sub(r"\bOUTROS\b", "Outros", var_aposentadoria)
    return var_aposentadoria

# Apply the 'renomear' function to the 'tipo_aposentadoria' column.
df['tipo_aposentadoria'] = df['tipo_aposentadoria'].apply(renomear)

# Convert the 'org_lotacao' column to title case.
df['org_lotacao'] = df['org_lotacao'].str.title()

# Define a function to perform specific corrections on organization names.
def renomear2(var_aposentadoria):
    var_aposentadoria = re.sub(r"\bMin Gestao E Inov Em Serv Publicos\b", "Ministério Gestão E Inovação Em Serviços Públicos", var_aposentadoria)
    var_aposentadoria = re.sub(r"\bMinisterio Dos Povos Indigenas\b", "Ministário dos Povos Indígenas", var_aposentadoria)
    var_aposentadoria = re.sub(r"\bMinisterio Da Previdencia Social\b", "Ministério Da Previdência Social", var_aposentadoria)
    var_aposentadoria = re.sub(r"\bMinisterio Do Planejamento E Orcamento\b", "Ministério Do Planejamento E Orçamento", var_aposentadoria)
    var_aposentadoria = re.sub(r"\bMin Da Integ E Do Desenv Regional\b", "Ministério da Integração E Do Desenvolvimento Regional", var_aposentadoria)
    var_aposentadoria = re.sub(r"\bMinisterio Do Trabalho E Previdencia\b", "Ministério do Trabalho E Previdência", var_aposentadoria)
    var_aposentadoria = re.sub(r"\bMin Do Desenv Agr E Agric Familiar\b", "Ministério do Desenvolvimento Agrário E Agricultura Familiar", var_aposentadoria)
    var_aposentadoria = re.sub(r"\bMin Desenvolv Ind Comercio E Servicos\b", "Ministério do Desenvolvimento, Indústria, Comércio E Serviços", var_aposentadoria)
    var_aposentadoria = re.sub(r"\bComunicacoes\b", "Comunicações", var_aposentadoria)
    var_aposentadoria = re.sub(r"\bMinisterio\b", "Ministério", var_aposentadoria)
    return var_aposentadoria

# Apply the 'renomear2' function to the 'org_lotacao' column.
df['org_lotacao'] = df['org_lotacao'].apply(renomear2)

# Define a function to format the 'regime_juridico' column.
def renomear3(var_regime):
    var_regime = re.sub(r"\bREGIME JURIDICO UNICO\b", "Regime Jurídico Único", var_regime)
    return var_regime

# Apply the 'renomear3' function to the 'regime_juridico' column.
df['regime_juridico'] = df['regime_juridico'].apply(renomear3)

# Define a dictionary to map specific ministries to broader thematic categories.
ministerios = {'Ministério Gestão E Inovação Em Serviços Públicos': 'Economia, Finanças, Planejamento, Previdência e Gestão',
                'Ministério Da Cultura': 'Cultura, Turismo e Esportes',
                'Ministério Da Saúde': 'Saúde',
                'Ministério Da Defesa': 'Justiça, Segurança e Órgãos de Controle e Defesa',
                'Ministério Da Educação': 'Educação e Ciência e Tecnologia',
                'Ministário dos Povos Indígenas': 'Desenvolvimento Social e Direitos Humanos',
                'Presidência Da República': 'Presidência da República',
                'Ministério De Minas E Energia': 'Infraestrutura, Transporte, Desenvolvimento Regional, Portos e Aeroportos, Minas e Energia',
                'Ministério Da Justiça E Segurança Pública': 'Justiça, Segurança e Órgãos de Controle e Defesa',
                'Ministério Da Ciência, Tecnologia, Inovações E Comunicações': 'Educação e Ciência e Tecnologia',
                'Ministério Dos Transportes': 'Infraestrutura, Transporte, Desenvolvimento Regional, Portos e Aeroportos, Minas e Energia',
                'Ministério Da Previdência Social': 'Economia, Finanças, Planejamento, Previdência e Gestão',
                'Ministério Do Planejamento E Orçamento': 'Economia, Finanças, Planejamento, Previdência e Gestão',
                'Ministério Da Fazenda': 'Economia, Finanças, Planejamento, Previdência e Gestão',
                'Ministério Das Relações Exteriores': 'Relações Exteriores',
                'Ministério Da Agricultura, Pecuária E Abastecimento': 'Meio ambiente, Agricultura e Agropecuária',
                'Ministério da Integração E Do Desenvolvimento Regional': 'Infraestrutura, Transporte, Desenvolvimento Regional, Portos e Aeroportos, Minas e Energia',
                'Ministério do Desenvolvimento Agrário E Agricultura Familiar': 'Meio ambiente, Agricultura e Agropecuária',
                'Ministério Do Meio Ambiente': 'Meio ambiente, Agricultura e Agropecuária',
                'Ministério Do Turismo': 'Cultura, Turismo e Esportes',
                'Ministério Do Trabalho E Emprego': 'Economia, Finanças, Planejamento, Previdência e Gestão',
                'Ministério De Portos E Aeroportos': 'Infraestrutura, Transporte, Desenvolvimento Regional, Portos e Aeroportos, Minas e Energia',
                'Ministério do Desenvolvimento, Indústria, Comércio E Serviços': 'Infraestrutura, Transporte, Desenvolvimento Regional, Portos e Aeroportos, Minas e Energia',
                'Controladoria-Geral Da União': 'Justiça, Segurança e Órgãos de Controle e Defesa',
                'Ministério Das Comunicações': 'Infraestrutura, Transporte, Desenvolvimento Regional, Portos e Aeroportos, Minas e Energia',
                'Ministério do Trabalho E Previdência': 'Economia, Finanças, Planejamento, Previdência e Gestão',
                'Defensoria Pública Da União': 'Justiça, Segurança e Órgãos de Controle e Defesa',
                'Ministério Da Economia': 'Economia, Finanças, Planejamento, Previdência e Gestão'
}

# Define a function to apply the categorization using the dictionary.
def categorizando(x):
    if x in ministerios:
        return ministerios[x]

# Create a new column 'categoria_ministerios' by applying the 'categorizando' function.
df['categoria_ministerios'] = df['org_lotacao'].apply(categorizando)

# Create a pivot table to aggregate data, calculating the mean contribution time and the count of records.
df1 = pd.pivot_table(df, values='tempo_contribuicao_servico_publico', index=['tipo_aposentadoria', 'decada_aposentadoria', 'categoria_ministerios'], aggfunc=['mean','count']).reset_index()

# Rename the aggregated columns for clarity.
df1 = df1.rename(columns={'mean':'media_contribuicao',
                          'count':'quantidade_vinculos'})

# Round the 'media_contribuicao' column to one decimal place.
df1['media_contribuicao'] = df1['media_contribuicao'].round(1) # Limiting the decimal places

# Upload

In [ ]:
# Import the bigquery library from google.cloud
from google.cloud import bigquery

# Define the schema for the destination BigQuery table.
# The schema is a list of SchemaField objects, where each object defines a column's:
# 1. Name (e.g., 'tipo_aposentadoria')
# 2. Data type (e.g., 'STRING')
# 3. Description (e.g., 'Tipo de aposentadoria')
schema=[bigquery.SchemaField('tipo_aposentadoria','STRING',description='Tipo de aposentadoria'),
        bigquery.SchemaField('decada_aposentadoria','INTEGER',description='Década da aposentadoria'),
        bigquery.SchemaField('categoria_ministerios','STRING',description='Categorias dos ministérios'),
        bigquery.SchemaField('media_contribuicao','FLOAT',description='Média do tempo de aposentadorias agrupadas por setor, década da aposentadoria e tipo da aposentadoria'),
        bigquery.SchemaField('quantidade_vinculos','INTEGER',description='Quantidade total de aposentadorias agrupadas por setor, década da aposentadoria e tipo da aposentadoria')
        ]

# Initialize the BigQuery client, specifying the Google Cloud project ID.
# This client object is the main entry point for interacting with the BigQuery API.
client = bigquery.Client(project='repositoriodedadosgpsp')

# Create a reference to the BigQuery dataset named 'perfil_remuneracao'.
# This object points to the dataset where the table will be created or updated.
dataset_ref = client.dataset('perfil_remuneracao')

# Create a reference to the target table within the dataset specified earlier.
# The table will be named 'SIAPE_ingresso_aposentadoria_media_v6'.
table_ref = dataset_ref.table('SIAPE_ingresso_aposentadoria_media_v6')

# Configure the load job by creating a LoadJobConfig object.
# Here, we specify the schema that BigQuery should use for the table.
job_config = bigquery.LoadJobConfig(schema=schema)

# Start the job to load data from the pandas DataFrame 'df1' into the specified BigQuery table ('table_ref').
# The job is configured with the previously defined 'job_config'.
job = client.load_table_from_dataframe(df1, table_ref, job_config=job_config)

# Wait for the load job to complete and retrieve its result.
# This line is blocking and will pause the script's execution until the data upload is finished.
job.result()